In [1]:
import os, json, pickle, csv, pandas
import numpy as np
from classDeclarations import file_data
from random import sample
MAX_SAMPLE_SIZE = 100

In [2]:
Path = "../../CDIACPub8"

imgs = []
csvs = []
pdfs = []

missing_count = 0
df = pandas.read_csv('../CDIACFileData/labels/cdiac_naivetruth_processed.csv')
for subdir, dirs, files in os.walk(Path):
    for file_name in files:
        file_path = os.path.abspath(os.path.join(subdir, file_name))
        if df['path'].str.contains(file_name).any():
            if file_path.endswith('.png') or file_path.endswith('.jpg'):
                imgs.append(file_path)
            elif file_path.endswith('.csv') or file_path.endswith('.tsv'):
                csvs.append(file_path)
            elif file_path.endswith('.pdf') or file_path.endswith('.txt'):
                pdfs.append(file_path)
        else:
            missing_count += 1


img_sample_names = sample(imgs, MAX_SAMPLE_SIZE)
csv_sample_names = sample(csvs, MAX_SAMPLE_SIZE)
pdfs_sample_names = sample(pdfs, MAX_SAMPLE_SIZE)

/home/cc/.local/lib/python3.8/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [3]:
print("loading files now...")
one_gram = json.load(open('../CDIACFileData/one_grams_distr_256.json', 'r'))
two_gram = json.load(open('../CDIACFileData/two_grams_distr_256.json', 'r'))

byte_distr = pandas.DataFrame.from_dict(one_gram)
two_grams_dicts = pandas.DataFrame.from_dict(two_gram)
#best_extractors = json.load(open("../CorrelatingExtractors/best_extractors.json"))
print("loading files done!")

loading files now...
loading files done!


In [ ]:
img_data = []
csv_data = []
pdf_data = []
dataset = [img_data, csv_data, pdf_data]
for idx, data_list in enumerate(dataset):
    curr_sample = []
    if idx == 0:
        curr_sample = img_sample_names
    elif idx == 1:
        curr_sample = csv_sample_names
    else:
        curr_sample = pdfs_sample_names
    
    for file_name in curr_sample:
        print(type(byte_distr[file_name]))
        print(type(two_grams_dicts[file_name]))

In [6]:
img_data = []
csv_data = []
pdf_data = []
dataset = [img_data, csv_data, pdf_data]
for idx, data_list in enumerate(dataset):
    curr_sample = []
    if idx == 0:
        curr_sample = img_sample_names
    elif idx == 1:
        curr_sample = csv_sample_names
    else:
        curr_sample = pdfs_sample_names
   
    for file_name in curr_sample:
        #best_extractor_row_index = df.path[df.path==file_name].index.tolist()
        if  file_name.endswith('.txt') or file_name.endswith('.pdf'):
            best_extractor = 'freetext'
        elif file_name.endswith('.csv') or file_name.endswith('.tsv'):
            best_extractor = 'tabular'
        elif file_name.endswith('.png') or file_name.endswith('.jpg'):
            best_extractor = 'image'
    

        curr_file_data = file_data(file_name, byte_distr[file_name], two_grams_dicts[file_name], best_extractor)
        data_list.append(curr_file_data)

/home/cc/CDIACPub8/oceans/NOAA_Workshop/McKinley_Data/co2flx_regions_detrend.txt
/home/cc/CDIACPub8/oceans/Moorings/Papa_145W_50N/README.txt
/home/cc/CDIACPub8/oceans/VOS_Colibri_Line/2016/35MJ20160304_metadata.pdf
/home/cc/CDIACPub8/oceans/VOS_Skaugran_Line/Data_final/SK16E.txt
/home/cc/CDIACPub8/oceans/VOS_Palmer_Lines/Palmer_2003/Palmer_methods.pdf
/home/cc/CDIACPub8/oceans/Bigelow_ECoast/HB2011_2012/hb1105_readme.txt
/home/cc/CDIACPub8/oceans/VOS_New_Century_2/2014/PANC20140612 _PI_OME.pdf
/home/cc/CDIACPub8/CdiacBundles/33WA/WS15099_Readme.pdf
/home/cc/CDIACPub8/oceans/Gordon_Gunter/2008/gu0805_leg2_readme.txt
/home/cc/CDIACPub8/oceans/VOS_New_Century_2/2014/PANC20140910_PI_OME.pdf
/home/cc/CDIACPub8/oceans/VOS_Colibri_Line/2010/Colibri_18oct22oct2010_Metadata.pdf
/home/cc/CDIACPub8/oceans/VOS_Monte_Rio_Lines/MR_2010/RioBlanco_17nov1dec2010_Metadata.pdf
/home/cc/CDIACPub8/oceans/Benjamin/For_Stew/CdiacBundles/49P1/Pyxis_metadata_NIES.pdf
/home/cc/CDIACPub8/oceans/VOS_Explorer_line

In [7]:
print('Dumping!')

with open('gathered_data_revised_256.pkl', 'wb+') as handle:
    pickle.dump(dataset, file=handle, protocol=pickle.HIGHEST_PROTOCOL)
print('Dumped.')



Dumping!
Dumped.
